In [ ]:
#import all modules
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import re

import mir3.modules.features as feat
import mir3.modules.tool.wav2spectrogram as spec
import mir3.modules.features.centroid as cent
import mir3.modules.features.rolloff as roll
import mir3.modules.features.flatness as flat
import mir3.modules.features.flux as flux
import mir3.modules.features.mfcc as mfcc
import mir3.modules.features.diff as diff
import mir3.modules.features.stats as stats
reload(stats)
import mir3.modules.features.join as join
import mir3.modules.tool.to_texture_window as tex

import copy

from scipy.stats import friedmanchisquare as friedman
from scipy.stats import wilcoxon as wilcoxon
from scipy.stats import ttest_ind as ttest
from scipy.stats import f_oneway as f_oneway
from sklearn.preprocessing import normalize
from sklearn import preprocessing
from sklearn.cross_validation import ShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
#from sklearn.cross_validation import StratifiedKFold
import hmmlearn
from hmmlearn.hmm import GaussianHMM, MultinomialHMM

import numpy as np
import time

from scipy.stats import friedmanchisquare as friedman
from scipy.stats import wilcoxon as wilcoxon
from scipy.stats import ttest_ind as ttest
from scipy.stats import f_oneway as f_oneway
import operator
from ipywidgets import FloatProgress
from IPython.display import display


def dataset_characterization(dataset_file, dataset_dir=""):
    dataset = {} # Dictionary index = filename, content = class
    with open(dataset_dir + dataset_file) as f:
        for line in f:
            p = re.split(" |,|\t", line.rstrip('\n').rstrip('\r'))
            #p = sorted(p)
            dataset[p[0]] = p[1]
            
    return sorted(dataset.items(), key=operator.itemgetter(0))

def dataset_class_histogram(dataset):
    histogram = {}
    for data in dataset:
        #print data[1]
        #print file_name
        if data[1] not in histogram:
            histogram[data[1]] = 1
        else:
            histogram[data[1]] += 1
    return histogram

dataset_dir = "./fernando_falas/"
dataset_file = "labels.txt"

dataset = dataset_characterization(dataset_file, dataset_dir)
print dataset_class_histogram(dataset)

dataset_dir2 = "./dataset_noise_f/noise_50/"
dataset_file2 = "label_noise_50.txt"

dataset2 = dataset_characterization(dataset_file2, dataset_dir2)
print dataset_class_histogram(dataset2)

dataset_dir3 = "./dataset_noise_f/noise_90/"
dataset_file3 = "label_noise_90.txt"

dataset3 = dataset_characterization(dataset_file3, dataset_dir3)
print dataset_class_histogram(dataset3)

def features_gtzan(filename, directory=""):
    # Calculate spectrogram (normalizes wavfile)
    converter = spec.Wav2Spectrogram()
    s = converter.convert(open(directory + filename), window_length=512, dft_length=512,
                window_step=256, spectrum_type='magnitude', save_metadata=True, wav_rate=16000)
    
    # Extract low-level features, derivatives, and run texture windows    
    
    d = diff.Diff()
    #features = (cent.Centroid(), roll.Rolloff(), flat.Flatness(), flux.Flux(), mfcc.Mfcc())
    features = (mfcc.Mfcc(),)
    all_feats = None
    for f in features:
        track = f.calc_track(s) # Feature track
        all_feats = join.Join().join([all_feats, track])
        dtrack = d.calc_track(track) # Differentiate
        all_feats = join.Join().join([all_feats, dtrack])
        ddtrack = d.calc_track(dtrack) # Differentiate again
        all_feats = join.Join().join([all_feats, ddtrack])    

        # Texture window
    #t = tex.ToTextureWindow().to_texture(all_feats, 40)
        
    # Statistics
    #s = stats.Stats()
    #d = s.stats([t], mean=True, variance=True)    
    return all_feats

def low_level_features(dataset_file, dataset_dir=""): # Estimate low-level features. 
                                                      # Returns sklearn-compatible structures.
    d = dataset_characterization(dataset_file, dataset_dir)
    #print d
    labels = []
    features = []
    progress = FloatProgress(min=0, max=len(d))
    display(progress)
    progress.value = 0
    for f in d:
        feat = features_gtzan(filename=f[0], directory=dataset_dir)
        if not np.any(np.isnan(feat.data)):
            features.append(feat.data)
            labels.append(f[1])
        progress.value += 1
    return features, labels

dataset = dataset_characterization(dataset_file, dataset_dir)
features, labels = low_level_features(dataset_file, dataset_dir)

dataset2 = dataset_characterization(dataset_file2, dataset_dir2)
features2, labels2 = low_level_features(dataset_file2, dataset_dir2)

dataset3 = dataset_characterization(dataset_file3, dataset_dir3)
features3, labels3 = low_level_features(dataset_file3, dataset_dir3)

/home/fernanda_local/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/fernanda_local/.local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


{'TRABALHO': 120, 'CASA': 120, 'CINEMA': 120, 'AEROPORTO': 120, 'HOSPITAL': 120, 'FARMACIA': 120, 'ESCOLA': 120, 'ACADEMIA': 120, 'MERCADO': 120, 'FACULDADE': 120}
{'TRABALHO': 120, 'CASA': 120, 'CINEMA': 120, 'AEROPORTO': 120, 'HOSPITAL': 120, 'FARMACIA': 120, 'ESCOLA': 120, 'ACADEMIA': 120, 'MERCADO': 120, 'FACULDADE': 120}
{'TRABALHO': 120, 'CASA': 120, 'CINEMA': 120, 'AEROPORTO': 120, 'HOSPITAL': 120, 'FARMACIA': 120, 'ESCOLA': 120, 'ACADEMIA': 120, 'MERCADO': 120, 'FACULDADE': 120}


In [ ]:
bases_teste = ["./fernando_falas/", "./dataset_noise_f/noise_10/", "./dataset_noise_f/noise_20/", "./dataset_noise_f/noise_30/", "./dataset_noise_f/noise_40/", "./dataset_noise_f/noise_50/","./dataset_noise_f/noise_60/", "./dataset_noise_f/noise_70/", "./dataset_noise_f/noise_80/", "./dataset_noise_f/noise_90/", "./dataset_noise_f/noise_100/"]
rotulos_teste = ["labels.txt","label_noise_10.txt","label_noise_20.txt", "label_noise_30.txt", "label_noise_40.txt", "label_noise_50.txt", "label_noise_60.txt", "label_noise_70.txt", "label_noise_80.txt", "label_noise_90.txt", "label_noise_100.txt"]
 
    
#bases_teste = ["./dataset_noise_f/noise_80/", "./dataset_noise_f/noise_90/", "./dataset_noise_f/noise_100/"] 
#rotulos_teste = ["label_noise_80.txt", "label_noise_90.txt", "label_noise_100.txt"]

       
    #Rodar todas as bases de teste
for rev in xrange(len(bases_teste)):

    #dowloads datasets
    
    dataset_dir4 = bases_teste[rev]
    dataset_file4 = rotulos_teste[rev]

    dataset4 = dataset_characterization(dataset_file4, dataset_dir4)
    print "base de teste, ", bases_teste[rev], ": ", dataset_class_histogram(dataset4)
        
    #Feature extraction
    dataset4 = dataset_characterization(dataset_file4, dataset_dir4)
    features4, labels4 = low_level_features(dataset_file4, dataset_dir4)
             
        
    #Classification
    def label_to_numbers(labels, d):
        return [d.keys().index(i) for i in labels]


    def numbers_to_labels(numbers, d):
        return [d.keys[i] for i in numbers]

    def f1_from_confusion(c):
        ret = []
        for i in xrange(c.shape[0]):
            n_i = np.sum(c[i,:])
            est_i = np.sum(c[:,i])
            if n_i > 0:
                R = c[i,i] / float(n_i)
            else:
                R = 0.0
            if est_i > 0:
                P = c[i,i] / float(est_i)
            else:
                P = 0.0

            if (R+P) > 0:
                F = 2*R*P/(R+P)
            else:
                F = 0.
            ret.append([R, P, F])
        return ret

    class MultipleHMM():
        base_model = None
        models = {}

        def __init__(self, base_model=None):
            self.base_model = base_model

        def fit(self, X, y, train_lengths):
            """Fits all internal models"""
            labels = set(y)
            for l in labels:
                print "Training label:", l
                l_index = [i for i in xrange(len(y)) if y[i] == l]
                if X[0].ndim == 1:
                    this_x = [X[i].reshape(-1, 1) for i in l_index]
                else:
                    this_x = [X[i] for i in l_index]

                this_lengths = [train_lengths[i] for i in l_index]

                my_x = this_x[0]
                for i in xrange(1, len(this_x)):
                    my_x = np.vstack ((my_x, np.array(this_x[i])))

                new_model = copy.deepcopy(self.base_model)
                new_model.fit(my_x, this_lengths)
                self.models[l] = new_model

        def predict(self, X):
            """Predicts a label for input X"""
            return_label = None
            best_prob = None
            for label in self.models.keys():

                #print "Computing probabilities on shape", X.shape
                if X.ndim == 1:
                    this_prob = self.models[label].score(X.reshape(-1,1))
                else:
                    this_prob = self.models[label].score(X)

                #print "Prob in label", label, "=", this_prob
                if (best_prob is None) or (this_prob > best_prob):
                    best_prob = this_prob
                    return_label = label
            return return_label
        
        
    #Statistical analysis
    from sklearn.model_selection import StratifiedKFold

    n_folds = 10
        
    def model_comparison(features, labels, features2, labels2, features3, labels3, features4, labels4, models): 
        #norm_features = normalize(features)
        features = np.array(features)
        features2 = np.array(features2)
        features3 = np.array(features3)
        features4 = np.array(features4)

        skf = StratifiedKFold(n_splits=n_folds)

        f1 = np.zeros((n_folds,len(models)))
        r = np.zeros((n_folds,len(models)))
        p = np.zeros((n_folds,len(models)))

        progress = FloatProgress(min=0, max=n_folds*len(models))
        display(progress)
        for m in xrange(len(models)):
            n = 0
            for train_index, test_index in skf.split(features, labels):
                #print( "TRAIN:", train_index, "\nTEST:", test_index)
                X_train1, X_train2, X_train3, X_test = [features[i] for i in train_index], [features2[i] for i in train_index], [features3[i] for i in train_index], [features4[i] for i in test_index]
                Y_train1, Y_train2, Y_train3, Y_test = [labels[i] for i in train_index], [labels2[i] for i in train_index], [labels3[i] for i in train_index], [labels4[i] for i in test_index]

                #X_train = np.vstack((X_train1, X_train2))
                X_train = np.concatenate((X_train1,X_train2,X_train3), axis=0)
                #X_train = X_train1 + X_train2
                Y_train = Y_train1 + Y_train2 + Y_train3
                
               

                all_trains = np.array(X_train[0])
                train_lengths = [X_train[0].shape[0]]
                for i in xrange(1, len(X_train)):
                    all_trains = np.vstack ((all_trains, np.array(X_train[i])))
                    train_lengths.append(X_train[i].shape[0])
                all_trains = all_trains    
                  
                scaler = preprocessing.StandardScaler().fit(all_trains)
                X_train = [scaler.transform(X_train[i]) for i in xrange(len(X_train))]
                #X_train2 = [scaler.transform(X_train2[i]) for i in xrange(len(X_train2))]
                X_test = [scaler.transform(X_test[i]) for i in xrange(len(X_test))]


                #cv = ShuffleSplit(len(X_train).shape[0], n_iter=10, test_size=0.2, random_state=0) # 80% train / 20% validation
                #classifier = GridSearchCV(estimator=copy.deepcopy(models[m]), cv=cv, param_grid=parameters_to_optimize[m], scoring='f1_weighted')
                classifier = models[m]

                classifier.fit(X_train, Y_train, train_lengths)# + train_lengths2)

                Y_pred = [classifier.predict(X_test[i]) for i in xrange(len(X_test))]
                #print Y_pred
                confusion = confusion_matrix(Y_test, Y_pred)
                print "confusion matrix:\n", confusion
                conf = f1_from_confusion(confusion)
                conf_all = np.average(conf, axis=0)
                print conf_all


                r[n,m] = conf_all[0]
                p[n,m] = conf_all[1]
                f1[n,m] = conf_all[2]
                n += 1

                progress.value += 1
        return r, p, f1
            
            
        
    import warnings
    warnings.filterwarnings('ignore')

    models = [MultipleHMM(GaussianHMM(n_components=12, covariance_type='diag', n_iter=50))]

    recall, precision, f1_score = model_comparison(features, label_to_numbers(labels, dataset_class_histogram(dataset)),features2, label_to_numbers(labels2, dataset_class_histogram(dataset2)),features3, label_to_numbers(labels3, dataset_class_histogram(dataset3)), features4, label_to_numbers(labels4, dataset_class_histogram(dataset4)), models)

    print "Teste: ", bases_teste[rev], "\nrecall:",recall, "\nprecision:", precision, "\nf1_score:", f1_score, "\n"
    print "media_recall:", np.average(recall, axis=0), "media_precision:", np.average(precision, axis=0), "media_f1_score:", np.average(f1_score, axis=0), "\n"

    sigma = np.std(f1_score)
    print "Desvio Padrao:", sigma, "\n"


    if len(models) > 2:
    #    print [f1_score[:,i].T for i in range(len(models))]
        print "Anova: ", f_oneway( *f1_score.T  )[1]

    elif len(models) == 2:
        print "T-test:", ttest( f1_score[:,0].T,  f1_score[:,1].T)[1] 

base de teste,  ./fernando_falas/ :  {'TRABALHO': 120, 'CASA': 120, 'CINEMA': 120, 'AEROPORTO': 120, 'HOSPITAL': 120, 'FARMACIA': 120, 'ESCOLA': 120, 'ACADEMIA': 120, 'MERCADO': 120, 'FACULDADE': 120}
Training label: 0
Training label: 1
Training label: 2
Training label: 3
Training label: 4
Training label: 5
Training label: 6
Training label: 7
Training label: 8
Training label: 9
confusion matrix:
[[ 0  9  0  1  0  0  0  0  2  0]
 [ 0 12  0  0  0  0  0  0  0  0]
 [ 0  5  3  0  0  1  0  0  2  1]
 [ 2  3  0  4  0  0  0  0  3  0]
 [ 1  8  0  0  1  0  2  0  0  0]
 [ 0  9  0  0  0  3  0  0  0  0]
 [ 0  9  0  0  0  0  3  0  0  0]
 [ 0  0  1  0  0  0  0  5  6  0]
 [ 1  8  0  0  0  0  0  0  3  0]
 [ 0  3  0  0  0  0  0  0  2  7]]
[ 0.34166667  0.61234848  0.35233032]
Training label: 0
Training label: 1
Training label: 2
Training label: 3
Training label: 4
Training label: 5
Training label: 6
Training label: 7
Training label: 8
Training label: 9
confusion matrix:
[[12  0  0  0  0  0  0  0  0  0]
 